In [1]:
# !jt -t chesterish -T -N -kl

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostRegressor, Pool, cv

from tqdm.notebook import tqdm

from joblib import Parallel, delayed
from bs4 import BeautifulSoup  as bs
from pprint import pprint   
import requests  
import json
from ast import literal_eval
import json

import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler

# import my_func
# import importlib
# importlib.reload(my_func)

# import huperopt_class
# importlib.reload(huperopt_class)

# from huperopt_class import HyperOpt

from IPython.display import Audio
from russian_names import RussianNames

from huperopt_class import HyperOpt

In [3]:
RANDOM_SEED = 42

In [4]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [5]:
data = pd.read_csv('data_for_stacking.csv')
cat_features_ids = ['bodyType', 'brand', 'color','descr_labels', 'fuelType', 
                    'model_name', 'vehicleTransmission',
                    'pts', 'privod', 'wheel', 'state', 'descr_labels']

In [6]:
X = data.query('sample == 1').drop(['sample'], axis=1)
y = X['price']

X.drop('price', axis=1, inplace=True, errors='ignore')
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

In [7]:
VAL_SIZE=0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [8]:
y_train = np.log(y_train)
y_test = np.log(y_test)

In [9]:
# CATBOOST

catboost_params = {
    'iterations': 5000,
    'learning_rate': 0.05775539388456,
    'depth': 12,
    'random_seed': RANDOM_SEED,
    'eval_metric': 'MAPE',
    'custom_metric': ['R2', 'MAE'],
    'l2_leaf_reg': 1,
    "loss_function": "MAPE"
}

ITERATIONS = 5000
LR         =  0.05775539388456 # 0.08343057119918101 #0.09399760402267439 #0.1
depth = 12 # 4 # 8 #12

In [ ]:
X_train.dublicates()

In [10]:
train_pool = Pool(
    X_train,
    y_train,
    cat_features=cat_features_ids,
)
test_pool = Pool(
    X_test,
    y_test,
    cat_features=cat_features_ids,
)

model = CatBoostRegressor(iterations = catboost_params['iterations'],
                          learning_rate = catboost_params['learning_rate'],
                          random_seed = RANDOM_SEED,
                          eval_metric=catboost_params['eval_metric'],
                          custom_metric=catboost_params['custom_metric'],
                          l2_leaf_reg=catboost_params['l2_leaf_reg'],
                          depth=catboost_params['depth'],
                          metric_period=catboost_params['depth'],
                          od_type='Iter',
                          od_wait=20,
                          rsm=0.2,
                          devices='GPU'
                         )
model.fit(train_pool,
         eval_set=test_pool,
         verbose_eval=catboost_params['depth']*4,
         use_best_model=True,
         plot=False
         )

0:	learn: 0.0703260	test: 0.0704674	best: 0.0704674 (0)	total: 386ms	remaining: 32m 8s
48:	learn: 0.0132364	test: 0.0134939	best: 0.0134939 (48)	total: 14.5s	remaining: 24m 24s
96:	learn: 0.0107103	test: 0.0111258	best: 0.0111258 (96)	total: 28.2s	remaining: 23m 45s


KeyboardInterrupt: 

In [ ]:
predict = model.predict(X_test)

predict = np.e ** predict
y_test = np.e ** y_test

# оцениваем точность
MAPE = f'{(mape(y_test, predict))*100:0.4f}'
print(f"Точность модели по метрике MAPE: {MAPE}%")

In [ ]:
data.duplicated(subset='sell_id', keep='first').value_counts()

In [ ]:
import hyperopt 
DEVICE='GPU'
catb_kwargs = {'iterations': 5000, 'X_train': X,'y_train': y,
               'categorical_features_indices': cat_features_ids,
               'ignored_features': 'price',
               'random_seed':RANDOM_SEED, 
               'eval_metric': 'MAPE',
               'od_type':'Iter', 
               'od_wait':20
               }

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 1e-1),
    'max_depth': hyperopt.hp.uniform('max_depth', 2, 16)}

rstate = np.random.RandomState(RANDOM_SEED)
trials = hyperopt.Trials()

catb_hyperopt_inst = HyperOpt(**catb_kwargs)


trials = hyperopt.Trials()
best = hyperopt.fmin(
    catb_hyperopt_inst.hyperopt_catb_score,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=2,
    trials=trials,
    rstate=rstate
)
print(best)

In [ ]:
import winsound
freq = 2000 # Set frequency To 2500 Hertz
dur = 700 # Set duration To 1000 ms == 1 second
winsound.Beep(freq, dur)